In [214]:
import pickle
import pandas as pd

class DataExtractor:

    def __init__(self, data_path, expired_path):
        self.data_path = data_path
        self.expired_path = expired_path
        self.data = self.load_data()
        self.df = None

    def load_data(self):
        with open(self.data_path, 'rb') as f:
            return pickle.load(f)

    def process_invoices(self):
        problem_id = []
        df_dict = {
            'invoice_id': [], 'created_on': [], 'invoiceitem_id': [], 
            'invoiceitem_name': [], 'type': [], 'unit_price': [], 'quantity': []
        }

        for i, invoice in enumerate(self.data):
            try:
                for item in invoice['items']:
                    df_dict['invoice_id'].append(invoice['id'])
                    df_dict['created_on'].append(invoice['created_on'])
                    df_dict['invoiceitem_id'].append(item['item']['id'])
                    df_dict['invoiceitem_name'].append(item['item']['name'])
                    df_dict['type'].append(item['item']['type'])
                    df_dict['unit_price'].append(item['item']['unit_price'])
                    df_dict['quantity'].append(item['quantity'])
            except KeyError:
                problem_id.append(invoice['id'])

        self.df = pd.DataFrame.from_dict(df_dict)
        self.clean_data()
        print(f"{len(problem_id)} empty invoices identified: {problem_id}")

    def clean_data(self):
        self.df['invoice_id'] = self.df['invoice_id'].apply(lambda number: int(number.replace('O', '0')) if isinstance(number, str) else int(number))
        self.df['quantity'] = self.df['quantity'].apply(lambda x: 10 if x == 'ten' else 5 if x == 'five' else x)
        self.df['type'] = self.df['type'].map({0: 'Material', 'O': 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'})
        self.df['total_price'] = self.df['unit_price'] * self.df['quantity']
        self.df['invoice_total'] = self.df['total_price'].groupby(self.df['invoice_id']).transform('sum')
        self.df['percentage_in_invoice'] = self.df['unit_price'] * self.df['quantity'] / self.df['invoice_total']
        self.df['created_on'] = pd.to_datetime(self.df['created_on'], format='mixed', errors='coerce')
        self.check_expired()
        self.sort_dataframe()

    def check_expired(self):
        with open(self.expired_path, 'r') as f:
            expired = f.read().split(', ')
        expired = [int(i) for i in expired]
        self.df['is_expired'] = self.df['invoice_id'].isin(expired)

    def sort_dataframe(self):
        self.df = self.df.sort_values(['invoice_id', 'invoiceitem_id'])
        self.df = self.df[['invoice_id','created_on', 'invoiceitem_id','invoiceitem_name', 'type', 'unit_price', 'total_price', 'percentage_in_invoice', 'is_expired']]

    def get_csv(self):
        self.process_invoices()
        self.df.to_csv('invoices.csv', index=False)

In [215]:
extractor = DataExtractor('data/invoices_new.pkl','data/expired_invoices.txt')

In [216]:
extractor.get_csv()

2 empty invoices identified: [379961, 350506]
